# 数据探索性分析

**项目**: 基于网络流量特征的钓鱼网站检测技术研究

**阶段**: 阶段一 - 数据采集与预处理

**日期**: 2025年12月

---

## 分析目标

1. 了解数据集基本情况（数据量、标签分布）
2. 分析URL长度分布（钓鱼 vs 正常）
3. 分析域名长度分布
4. 分析特殊字符频率
5. 分析数据来源分布
6. 为后续特征提取提供参考

In [ ]:
# Cell 1: 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.parse import urlparse
import os
import sys

# 添加项目根目录到路径
sys.path.append('..')
from src.data_loader import DataLoader
from config import PROCESSED_DATA_DIR

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置绘图风格
sns.set_style('whitegrid')
plt.style.use('seaborn-v0_8-whitegrid')

# 创建图表保存目录
figures_dir = os.path.join(PROCESSED_DATA_DIR, 'figures')
os.makedirs(figures_dir, exist_ok=True)

print(f"图表保存目录: {figures_dir}")
print("库导入完成!")

## 1. 数据加载与基本信息

In [ ]:
# Cell 2: 加载数据
loader = DataLoader()
df = loader.load_dataset()

# 打印数据集基本信息
loader.print_info()

print(f"\n数据列: {list(df.columns)}")
print(f"\n数据类型:")
print(df.dtypes)
print(f"\n前5条数据:")
df.head()

In [ ]:
# Cell 3: 数据质量检查
print("=" * 50)
print("数据质量检查")
print("=" * 50)

print(f"\n1. 总样本数: {len(df)}")
print(f"2. 重复URL数量: {df['url'].duplicated().sum()}")
print(f"3. 空值统计:")
print(df.isnull().sum())
print(f"\n4. 唯一值统计:")
print(f"   - 唯一URL数: {df['url'].nunique()}")
print(f"   - 唯一域名数: {df['domain'].nunique()}")
print(f"   - 数据来源数: {df['source'].nunique()}")

## 2. 标签分布分析

In [ ]:
# Cell 4: 标签分布可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 柱状图
labels = ['正常网站', '钓鱼网站']
counts = df['label'].value_counts().sort_index()
colors = ['#2ecc71', '#e74c3c']

bars = axes[0].bar(labels, counts.values, color=colors, edgecolor='black', linewidth=1.2)
axes[0].set_ylabel('数量', fontsize=12)
axes[0].set_title('数据集标签分布', fontsize=14, fontweight='bold')

# 添加数值标签
for bar, count in zip(bars, counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
                 f'{count}', ha='center', va='bottom', fontsize=12, fontweight='bold')

# 饼图
axes[1].pie(counts.values, labels=labels, autopct='%1.1f%%',
            colors=colors, explode=(0.02, 0.02), shadow=True,
            textprops={'fontsize': 12})
axes[1].set_title('标签比例', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, '01_label_distribution.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n标签分布统计:")
print(f"- 正常网站: {counts[0]} ({counts[0]/len(df)*100:.1f}%)")
print(f"- 钓鱼网站: {counts[1]} ({counts[1]/len(df)*100:.1f}%)")
print(f"- 类别平衡比: {counts[0]/counts[1]:.2f}")

## 3. URL长度分析

In [ ]:
# Cell 5: URL长度分析
df['url_length'] = df['url'].str.len()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 分布直方图
for label, color, name in [(0, '#2ecc71', '正常'), (1, '#e74c3c', '钓鱼')]:
    subset = df[df['label'] == label]['url_length']
    axes[0].hist(subset, bins=50, alpha=0.6, label=name, color=color, edgecolor='white')

axes[0].set_xlabel('URL长度', fontsize=12)
axes[0].set_ylabel('频数', fontsize=12)
axes[0].set_title('URL长度分布对比', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].axvline(df[df['label']==0]['url_length'].mean(), color='#27ae60', linestyle='--', label='正常均值')
axes[0].axvline(df[df['label']==1]['url_length'].mean(), color='#c0392b', linestyle='--', label='钓鱼均值')

# 箱线图
box_data = [df[df['label']==0]['url_length'], df[df['label']==1]['url_length']]
bp = axes[1].boxplot(box_data, labels=['正常', '钓鱼'], patch_artist=True)
bp['boxes'][0].set_facecolor('#2ecc71')
bp['boxes'][1].set_facecolor('#e74c3c')
axes[1].set_xlabel('类别', fontsize=12)
axes[1].set_ylabel('URL长度', fontsize=12)
axes[1].set_title('URL长度箱线图', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, '02_url_length.png'), dpi=150, bbox_inches='tight')
plt.show()

# 打印统计信息
print("\nURL长度统计:")
print(df.groupby('label')['url_length'].describe().round(2))

## 4. 域名长度分析

In [ ]:
# Cell 6: 域名长度分析
df['domain_length'] = df['domain'].str.len()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 分布直方图
for label, color, name in [(0, '#2ecc71', '正常'), (1, '#e74c3c', '钓鱼')]:
    subset = df[df['label'] == label]['domain_length']
    axes[0].hist(subset, bins=30, alpha=0.6, label=name, color=color, edgecolor='white')

axes[0].set_xlabel('域名长度', fontsize=12)
axes[0].set_ylabel('频数', fontsize=12)
axes[0].set_title('域名长度分布对比', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)

# 箱线图
box_data = [df[df['label']==0]['domain_length'], df[df['label']==1]['domain_length']]
bp = axes[1].boxplot(box_data, labels=['正常', '钓鱼'], patch_artist=True)
bp['boxes'][0].set_facecolor('#2ecc71')
bp['boxes'][1].set_facecolor('#e74c3c')
axes[1].set_xlabel('类别', fontsize=12)
axes[1].set_ylabel('域名长度', fontsize=12)
axes[1].set_title('域名长度箱线图', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, '03_domain_length.png'), dpi=150, bbox_inches='tight')
plt.show()

# 打印统计信息
print("\n域名长度统计:")
print(df.groupby('label')['domain_length'].describe().round(2))

## 5. 特殊字符分析

In [ ]:
# Cell 7: 特殊字符统计函数
def count_special_chars(url):
    """统计URL中的特殊字符数量"""
    return {
        'dots': url.count('.'),
        'hyphens': url.count('-'),
        'underscores': url.count('_'),
        'slashes': url.count('/'),
        'at_symbol': url.count('@'),
        'digits': sum(c.isdigit() for c in url),
        'question_marks': url.count('?'),
        'equals': url.count('=')
    }

# 计算特殊字符
char_stats = df['url'].apply(lambda x: pd.Series(count_special_chars(x)))
df_analysis = pd.concat([df, char_stats], axis=1)

print("特殊字符统计完成!")
print(f"新增特征列: {list(char_stats.columns)}")

In [ ]:
# Cell 8: 特殊字符分布可视化
features = ['dots', 'hyphens', 'slashes', 'digits']
feature_names = ['点号(.)', '连字符(-)', '斜杠(/)', '数字']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, (feature, fname) in enumerate(zip(features, feature_names)):
    ax = axes[idx // 2, idx % 2]
    for label, color, name in [(0, '#2ecc71', '正常'), (1, '#e74c3c', '钓鱼')]:
        subset = df_analysis[df_analysis['label'] == label][feature]
        ax.hist(subset, bins=20, alpha=0.6, label=name, color=color, edgecolor='white')
    ax.set_xlabel(f'{fname}数量', fontsize=11)
    ax.set_ylabel('频数', fontsize=11)
    ax.set_title(f'{fname}分布对比', fontsize=12, fontweight='bold')
    ax.legend(fontsize=10)

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, '04_special_chars.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 9: 特殊字符统计对比
print("\n" + "=" * 60)
print("特殊字符统计对比 (平均值)")
print("=" * 60)

char_features = ['dots', 'hyphens', 'underscores', 'slashes', 'at_symbol', 'digits']
char_names = ['点号', '连字符', '下划线', '斜杠', '@符号', '数字']

comparison = pd.DataFrame()
comparison['特征'] = char_names
comparison['正常网站'] = [df_analysis[df_analysis['label']==0][f].mean() for f in char_features]
comparison['钓鱼网站'] = [df_analysis[df_analysis['label']==1][f].mean() for f in char_features]
comparison['差异'] = comparison['钓鱼网站'] - comparison['正常网站']

print(comparison.round(2).to_string(index=False))

## 6. 数据来源分布

In [ ]:
# Cell 10: 数据来源分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

source_counts = df['source'].value_counts()
colors_source = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12'][:len(source_counts)]

# 饼图
axes[0].pie(source_counts.values, labels=source_counts.index, autopct='%1.1f%%',
            colors=colors_source, explode=[0.02]*len(source_counts), shadow=True,
            textprops={'fontsize': 11})
axes[0].set_title('数据来源分布', fontsize=14, fontweight='bold')

# 按来源和标签分组的柱状图
source_label = df.groupby(['source', 'label']).size().unstack(fill_value=0)
source_label.columns = ['正常', '钓鱼']
source_label.plot(kind='bar', ax=axes[1], color=['#2ecc71', '#e74c3c'], edgecolor='black')
axes[1].set_xlabel('数据来源', fontsize=12)
axes[1].set_ylabel('数量', fontsize=12)
axes[1].set_title('各来源的标签分布', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, '05_source_distribution.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\n数据来源统计:")
print(source_label)

## 7. Top域名分析

In [ ]:
# Cell 11: Top域名分析
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 正常网站Top 10域名
normal_domains = df[df['label']==0]['domain'].value_counts().head(10)
axes[0].barh(normal_domains.index[::-1], normal_domains.values[::-1], color='#2ecc71', edgecolor='black')
axes[0].set_xlabel('数量', fontsize=12)
axes[0].set_title('正常网站 Top 10 域名', fontsize=14, fontweight='bold')

# 钓鱼网站Top 10域名
phishing_domains = df[df['label']==1]['domain'].value_counts().head(10)
axes[1].barh(phishing_domains.index[::-1], phishing_domains.values[::-1], color='#e74c3c', edgecolor='black')
axes[1].set_xlabel('数量', fontsize=12)
axes[1].set_title('钓鱼网站 Top 10 域名', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, '06_top_domains.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\n正常网站 Top 10 域名:")
print(normal_domains.to_string())
print("\n钓鱼网站 Top 10 域名:")
print(phishing_domains.to_string())

## 8. 特征相关性热力图

In [ ]:
# Cell 12: 特征相关性分析
# 选择数值特征
numeric_features = ['url_length', 'domain_length', 'dots', 'hyphens', 
                    'underscores', 'slashes', 'digits', 'label']
corr_matrix = df_analysis[numeric_features].corr()

# 绘制热力图
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlGn_r', center=0,
            square=True, linewidths=0.5, fmt='.2f',
            cbar_kws={'shrink': 0.8})
ax.set_title('特征相关性热力图', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, '07_correlation_heatmap.png'), dpi=150, bbox_inches='tight')
plt.show()

# 与标签的相关性
print("\n各特征与标签(label)的相关性:")
label_corr = corr_matrix['label'].drop('label').sort_values(key=abs, ascending=False)
print(label_corr.round(3).to_string())

## 9. 数据集总结

In [ ]:
# Cell 13: 总结报告
print("\n" + "=" * 70)
print("数据探索分析总结报告")
print("=" * 70)

print(f"\n【1. 数据集规模】")
print(f"    总样本数: {len(df)} 条")
print(f"    训练集: {int(len(df)*0.8)} 条 (80%)")
print(f"    测试集: {int(len(df)*0.2)} 条 (20%)")

print(f"\n【2. 类别平衡】")
print(f"    正常网站: {(df['label']==0).sum()} ({(df['label']==0).sum()/len(df)*100:.1f}%)")
print(f"    钓鱼网站: {(df['label']==1).sum()} ({(df['label']==1).sum()/len(df)*100:.1f}%)")
print(f"    类别比例: 1:1 (平衡)")

print(f"\n【3. URL特征统计】")
print(f"    正常URL平均长度: {df[df['label']==0]['url_length'].mean():.1f} 字符")
print(f"    钓鱼URL平均长度: {df[df['label']==1]['url_length'].mean():.1f} 字符")
print(f"    长度差异: {df[df['label']==1]['url_length'].mean() - df[df['label']==0]['url_length'].mean():.1f} 字符")

print(f"\n【4. 域名特征统计】")
print(f"    正常域名平均长度: {df[df['label']==0]['domain_length'].mean():.1f} 字符")
print(f"    钓鱼域名平均长度: {df[df['label']==1]['domain_length'].mean():.1f} 字符")

print(f"\n【5. 数据来源】")
for source, count in df['source'].value_counts().items():
    print(f"    {source}: {count} ({count/len(df)*100:.1f}%)")

print(f"\n【6. 初步观察结论】")
print("    1. 数据集类别平衡,适合直接进行模型训练")
print("    2. 钓鱼URL通常比正常URL更长,可作为重要特征")
print("    3. 钓鱼URL包含更多特殊字符(点号、斜杠、数字等)")
print("    4. 域名长度在两类之间存在差异,具有区分能力")
print("    5. 数据来源明确,钓鱼数据来自PhishTank,正常数据来自Tranco")

print(f"\n【7. 后续工作建议】")
print("    1. 进行30维特征提取(URL词法、TLS、HTTP、DNS)")
print("    2. 特征标准化处理")
print("    3. 使用RandomForest+XGBoost集成模型训练")

print("\n" + "=" * 70)
print("分析完成!图表已保存至: " + figures_dir)
print("=" * 70)

In [ ]:
# Cell 14: 保存分析结果
import json

# 生成分析报告JSON
analysis_report = {
    'dataset_info': {
        'total_samples': int(len(df)),
        'train_samples': int(len(df) * 0.8),
        'test_samples': int(len(df) * 0.2),
        'phishing_count': int((df['label']==1).sum()),
        'normal_count': int((df['label']==0).sum()),
        'balance_ratio': 1.0
    },
    'url_statistics': {
        'normal_avg_length': float(df[df['label']==0]['url_length'].mean()),
        'phishing_avg_length': float(df[df['label']==1]['url_length'].mean()),
        'normal_max_length': int(df[df['label']==0]['url_length'].max()),
        'phishing_max_length': int(df[df['label']==1]['url_length'].max())
    },
    'domain_statistics': {
        'normal_avg_length': float(df[df['label']==0]['domain_length'].mean()),
        'phishing_avg_length': float(df[df['label']==1]['domain_length'].mean())
    },
    'sources': df['source'].value_counts().to_dict(),
    'analysis_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}

# 保存报告
report_path = os.path.join(figures_dir, 'analysis_report.json')
with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(analysis_report, f, ensure_ascii=False, indent=2)

print(f"分析报告已保存至: {report_path}")
print("\n报告内容:")
print(json.dumps(analysis_report, ensure_ascii=False, indent=2))